In [38]:
import pandas as pd
import requests
import psycopg2


ids = {'WM Phoenix Open': 'R2024003','The Genesis Invitational': 'R2024007',
       'Mexico Open at Vidanta':'R2024540','Cognizant Classic':'R2024010',
       'Arnold Palmer Invitational':'R2024009', 
       'THE PLAYERS Championship':'R2024011', "Texas Children's Houston Open": 'R2024020',
       'Valspar Championship':'R2024475', 'The Valero Texas Open': 'R2023041',
       'RBC Heritage' : 'R2024012', 'THE CJ CUP Byron Nelson': 'R2024019',
       'Wells Fargo Championship': 'R2024480', 'Charles Schwab Challenge': 'R2024021',
        'RBC Canadian Open' : 'R2024032', 'the Memorial Tournament presented by Workday': 'R2024023' }

r1 = []
r2 = []
r3 = []
r4 = []
player_name = []
fedex_cup_points = []
official_money = []
position = []
score = []
tournament_name = []
tournament_id = []

for i,j in ids.items():
    payload = {"operationName":"TournamentPastResults","variables":{"tournamentPastResultsId":j,"year":None},"query":"query TournamentPastResults($tournamentPastResultsId: ID!, $year: Int) {\n  tournamentPastResults(id: $tournamentPastResultsId, year: $year) {\n    id\n    teams {\n      teamId\n      position\n      players {\n        id\n        firstName\n        lastName\n        shortName\n        displayName\n        abbreviations\n        abbreviationsAccessibilityText\n        amateur\n        country\n        countryFlag\n        lineColor\n        seed\n        status\n      }\n      rounds {\n        score\n        parRelativeScore\n      }\n      additionalData\n      total\n      parRelativeScore\n    }\n    rounds\n    additionalDataHeaders\n    availableSeasons {\n      year\n      displaySeason\n    }\n    winner {\n      id\n      firstName\n      lastName\n      totalStrokes\n      totalScore\n      countryFlag\n      countryName\n      purse\n      points\n      seed\n      displayPoints\n      displayPurse\n    }\n    winningTeam {\n      id\n      firstName\n      lastName\n      totalStrokes\n      totalScore\n      countryFlag\n      countryName\n      purse\n      points\n      seed\n      displayPoints\n      displayPurse\n    }\n    players {\n      id\n      position\n      player {\n        id\n        firstName\n        lastName\n        shortName\n        displayName\n        abbreviations\n        abbreviationsAccessibilityText\n        amateur\n        country\n        countryFlag\n        lineColor\n        seed\n        status\n      }\n      rounds {\n        score\n        parRelativeScore\n      }\n      additionalData\n      total\n      parRelativeScore\n    }\n  }\n}"}

    url = 'https://orchestrator.pgatour.com/graphql'

    response = requests.post(url, json = payload, headers = {'X-Api-Key':
    'da2-gsrx5bibzbb4njvhl7t37wqyl4'})

    results = response.json()


    for player in results['data']['tournamentPastResults']['players']:
        try:
            r3.append(player['rounds'][2]['score'])
            r4.append(player['rounds'][3]['score'])
            player_name.append(player['player']['displayName'])
            fedex_cup_points.append(player['additionalData'][0])
            official_money.append(player['additionalData'][1])
            position.append(player['position'])
            score.append(player['parRelativeScore'])
            tournament_name.append(j) 
            tournament_id.append(i) 
            r1.append(player['rounds'][0]['score'])
            r2.append(player['rounds'][1]['score'])
            
        except:
            break

data = {'tournament_name': tournament_id,
        'tournament_id' : tournament_name,
        'player_name' : player_name,
        'position' : position,
        'score' : score,
        'r1' : r1,
        'r2' : r2,
        'r3' : r3,
        'r4' : r4,
        'fedex_cup_points' : fedex_cup_points,
        'official_money' : official_money,
        }

df = pd.DataFrame(data)
cleaned = []

for a in df['official_money']:
    cleaned.append(a.split('$')[1].replace(',',''))

df['official_money'] = cleaned


df

,tournament_name,tournament_id,player_name,position,score,r1,r2,r3,r4,fedex_cup_points,official_money
0,WM Phoenix Open,R2024003,Nick Taylor,1,-21,60,70,68,65,500.000,1584000.00
1,WM Phoenix Open,R2024003,Charley Hoffman,2,-21,67,68,64,64,300.000,959200.00
2,WM Phoenix Open,R2024003,Sam Burns,T3,-18,68,67,67,64,162.500,519200.00
3,WM Phoenix Open,R2024003,Scottie Scheffler,T3,-18,68,66,66,66,162.500,519200.00
4,WM Phoenix Open,R2024003,Sahith Theegala,5,-17,65,64,69,69,110.000,360800.00
...,...,...,...,...,...,...,...,...,...,...,...
1020,the Memorial Tournament presented by Workday,R2024023,Eric Cole,T45,+10,70,75,74,79,15.375,60500.00
1021,the Memorial Tournament presented by Workday,R2024023,Shane Lowry,49,+12,74,73,68,85,13.500,54000.00
1022,the Memorial Tournament presented by Workday,R2024023,Cameron Young,T50,+13,76,72,76,77,12.750,51500.00
1023,the Memorial Tournament presented by Workday,R2024023,Cam Davis,T50,+13,73,72,77,79,12.750,51500.00


In [39]:
import psycopg2

db_creds = pd.read_csv('/users/zack burnside/desktop/database_creds.csv')

conn = psycopg2.connect(
    dbname=db_creds['dbname'][0],
    user=db_creds['user'][0],
    password=db_creds['password'][0],
    host=db_creds['host'][0],
    port="5433"
)

# Create a cursor
cur = conn.cursor()
cur.execute("TRUNCATE TABLE tournaments_results")

# Define the insert query
insert_query = """
    INSERT INTO public.tournaments_results (tournament_name, tournament_id, player_name, position, score, 
                                    r1, r2, r3, r4, fedex_cup_points, official_money)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
    
"""

# Assuming you have a DataFrame named df with the required columns
# Execute the insert query for each row of data
for index, row in df.iterrows():  # Iterate over DataFrame rows
    cur.execute(insert_query, row.tolist())

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

print('Data Inserted Into Table, Duplicates Ignored')


Data Inserted Into Table, Duplicates Ignored


In [42]:
conn = psycopg2.connect(
    dbname=db_creds['dbname'][0],
    user=db_creds['user'][0],
    password=db_creds['password'][0],
    host=db_creds['host'][0],
    port="5433"
)
cur = conn.cursor()
create_table_query = """
CREATE TABLE IF NOT EXISTS public.weekly AS
SELECT
    up.tournament_name, 
    up.user_name, 
    tr.official_money AS prize_money
FROM 
    public.user_picks up
JOIN 
    public.tournaments_results tr 
ON 
    up.tournament_name = tr.tournament_name 
    AND up.player_pick = tr.player_name;
"""

# Execute the SQL query to create the table
cur.execute("DROP TABLE IF EXISTS public.weekly")  # Drop the table if it already exists
cur.execute(create_table_query)  # Create the 'weekly' table

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

print('Weekly Table Created')

Weekly Table Created
